In [1]:
%run ../load_files.ipynb

In [2]:
%run ../data_all_matrixes.ipynb

,profile,age,gender,MS_HV,MS_type,EDSS,disease_duration,education,connectivity,degree,...,MFIS,SF36_mental,SF36_physical,DTI_FA,DTI_L1,DTI_MD,DTI_RX,RAW,FUNC,LS
ID,,,,,,,,,,,,,,,,,,,,,
FIS_001,none,45.506,female,MS,RR,1.5,16.772,secondary school,****,35.842105,...,8.0,42.97,54.45,"[[0.0, 0.439328825873252, 0.4625400010374732, ...","[[0.0, 0.001062246374944052, 0.001097384692440...","[[0.0, 0.000705840751585395, 0.000715541904089...","[[0.0, 0.0005276379438376123, 0.00052462053555...","[[0.0, 0.03552844351216702, 0.0006652241379404...",NaN,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
FIS_002,none,42.981,male,MS,RR,1.5,10.872,primary school,****,30.368421,...,4.0,60.25,53.15,"[[0.0, 0.4382497080416883, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001055207424665258, 0.0, 0.0, 0.0, 0....","[[0.0, 0.000703053658342256, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0005269767649653327, 0.0, 0.0, 0.0, 0...","[[0.0, 0.028578488896818185, 0.0, 0.0, 0.0, 0....",NaN,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
FIS_004,none,52.239,female,MS,SP,6.0,21.828,graduate,****,30.736842,...,12.0,56.38,39.40,"[[0.0, 0.3322280520926731, 0.0, 0.0, 0.0, 0.37...","[[0.0, 0.001171813137463012, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0008873735393089818, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0007451537309838455, 0.0, 0.0, 0.0, 0...","[[0.0, 0.06991186050430216, 0.0, 0.0, 0.0, 0.0...",NaN,"[[0.0, 1009.125, 0.0, 0.0, 0.0, 421.875, 0.0, ..."
FIS_005,none,36.672,male,MS,RR,1.0,11.619,post-graduate,****,27.736842,...,0.0,59.19,55.81,"[[0.0, 0.4078810391594139, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001044529380979277, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0007169208696827965, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0005531166160359597, 0.0, 0.0, 0.0, 0...","[[0.0, 0.02578800302151145, 0.0, 0.0, 0.0, 0.0...",NaN,"[[0.0, 20.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
FIS_007,attention,31.233,female,MS,RR,2.0,13.928,post-graduate,****,30.184211,...,5.0,47.17,49.24,"[[0.0, 0.4314795444662643, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.001069484424799801, 0.0, 0.0, 0.0, 0....","[[0.0, 0.0007114571565313628, 0.0, 0.0, 0.0, 0...","[[0.0, 0.0005324435254387877, 0.0, 0.0, 0.0, 0...","[[0.0, 0.035325089806942316, 0.0, 0.0, 0.0, 0....",NaN,"[[0.0, 60.75, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."


(98, 39)


In [3]:
%run ../y_multilevel.ipynb

,attention,memory
ID,,
FIS_001,0,0
FIS_002,0,0
FIS_004,0,0
FIS_005,0,0
FIS_007,1,0


(98, 2)

In [4]:
y = y_values.memory

In [5]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.333, random_state=2018, stratify=y)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(65, 39)
(33, 39)
(65,)
(33,)


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    ('reduce_dim', PCA()),
    ('classify', GaussianNB())
])

In [8]:
# Parametrized values
N_FEATURES_OPTIONS = range(1, 500, 10)
MATRIXES = list(all_combinations(column_matrixes))
EXTRA_1D_COLUMS = [None] # + list(all_combinations(available_columns))                            

param_grid = { 
        'combining__columns': MATRIXES,
        'filter__columns': EXTRA_1D_COLUMS,
        
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
}


In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [10]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] reduce_dim__n_components=431, filter__columns=None, combining__columns=('DTI_MD', 'RAW', 'LS') 
[CV] reduce_dim__n_components=431, filter__columns=None, combining__columns=('DTI_MD', 'RAW', 'LS') 
[CV] reduce_dim__n_components=431, filter__columns=None, combining__columns=('DTI_MD', 'RAW', 'LS') 
[CV] reduce_dim__n_components=71, filter__columns=None, combining__columns=('DTI_FA', 'RAW', 'LS') 
[CV] reduce_dim__n_components=71, filter__columns=None, combining__columns=('DTI_FA', 'RAW', 'LS') 
[CV] reduce_dim__n_components=71, filter__columns=None, combining__columns=('DTI_FA', 'RAW', 'LS') 
[CV] reduce_dim__n_components=311, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'RAW', 'LS') 
[CV] reduce_dim__n_components=311, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'RAW', 'LS') 
[CV]  reduce_dim__n_components=431, filter__columns=None, combining__col

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   10.1s


[CV] reduce_dim__n_components=141, filter__columns=None, combining__columns=('DTI_RX', 'LS') 
[CV]  reduce_dim__n_components=241, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD'), score=0.6666666666666666, total=   2.5s
[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'RAW', 'LS') 
[CV]  reduce_dim__n_components=241, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD'), score=0.5454545454545454, total=   2.6s
[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'RAW', 'LS') 
[CV]  reduce_dim__n_components=311, filter__columns=None, combining__columns=('DTI_L1', 'LS'), score=0.5454545454545454, total=   2.7s
[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_FA', 'DTI_MD', 'RAW', 'LS') 
[CV]  reduce_dim__n_components=311, filter__columns=None, combining__columns=('DTI_L1', 'LS'), score=0.6363636363636364, total=   2.8s
[CV] 

[CV]  reduce_dim__n_components=141, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'LS'), score=0.5454545454545454, total=   2.6s
[CV] reduce_dim__n_components=141, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'RAW') 
[CV]  reduce_dim__n_components=141, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'LS'), score=0.6363636363636364, total=   3.1s
[CV] reduce_dim__n_components=141, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'RAW') 
[CV]  reduce_dim__n_components=41, filter__columns=None, combining__columns=('DTI_FA', 'LS'), score=0.5454545454545454, total=   3.3s
[CV]  reduce_dim__n_components=41, filter__columns=None, combining__columns=('DTI_FA', 'LS'), score=0.6363636363636364, total=   2.7s
[CV] reduce_dim__n_components=141, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'RAW') 
[CV] reduce_dim__n_components=251, filter__columns=None, combining__columns=('DTI_RX', 'LS') 
[CV]  reduce_dim__n_components=41

[CV]  reduce_dim__n_components=491, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX', 'LS'), score=0.6363636363636364, total=   3.0s
[CV] reduce_dim__n_components=361, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'RAW') 
[CV] reduce_dim__n_components=361, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'RAW') 
[CV]  reduce_dim__n_components=341, filter__columns=None, combining__columns=('DTI_MD', 'DTI_RX', 'RAW'), score=0.5238095238095238, total=   3.4s
[CV] reduce_dim__n_components=431, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX', 'LS') 
[CV]  reduce_dim__n_components=491, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_RX', 'LS'), score=0.6666666666666666, total=   3.1s
[CV] reduce_dim__n_components=431, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'DTI_RX', 'LS') 
[CV]  reduce_dim__n_components=331, filter__columns=None, combinin

...

[CV] reduce_dim__n_components=171, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX', 'LS') 
[CV]  reduce_dim__n_components=301, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'RAW'), score=0.5454545454545454, total=   2.6s
[CV] reduce_dim__n_components=281, filter__columns=None, combining__columns=('DTI_FA', 'RAW', 'LS') 
[CV]  reduce_dim__n_components=301, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'RAW'), score=0.6666666666666666, total=   3.1s
[CV] reduce_dim__n_components=281, filter__columns=None, combining__columns=('DTI_FA', 'RAW', 'LS') 
[CV]  reduce_dim__n_components=331, filter__columns=None, combining__columns=('DTI_FA', 'RAW', 'LS'), score=0.5454545454545454, total=   2.9s
[CV] reduce_dim__n_components=281, filter__columns=None, combining__columns=('DTI_FA', 'RAW', 'LS') 
[CV]  reduce_dim__n_components=331, filter__columns=None, combining__columns=('DTI_FA', 'RAW', 'LS'), score=0.6363636363636364, total=   3.

[CV]  reduce_dim__n_components=361, filter__columns=None, combining__columns=('DTI_L1', 'LS'), score=0.6363636363636364, total=   1.9s
[CV] reduce_dim__n_components=421, filter__columns=None, combining__columns=('DTI_MD',) 
[CV]  reduce_dim__n_components=361, filter__columns=None, combining__columns=('DTI_L1', 'LS'), score=0.6666666666666666, total=   2.5s
[CV] reduce_dim__n_components=421, filter__columns=None, combining__columns=('DTI_MD',) 
[CV]  reduce_dim__n_components=241, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'LS'), score=0.5454545454545454, total=   3.0s
[CV] reduce_dim__n_components=421, filter__columns=None, combining__columns=('DTI_MD',) 
[CV]  reduce_dim__n_components=241, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'LS'), score=0.6363636363636364, total=   2.4s
[CV] reduce_dim__n_components=231, filter__columns=None, combining__columns=('DTI_L1', 'DTI_RX', 'LS') 
[CV]  reduce_dim__n_components=231, filter__colum

[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'RAW') 
[CV]  reduce_dim__n_components=231, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), score=0.5454545454545454, total=   1.9s
[CV]  reduce_dim__n_components=481, filter__columns=None, combining__columns=('DTI_L1', 'DTI_MD', 'LS'), score=0.6666666666666666, total=   2.4s
[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'RAW') 
[CV] reduce_dim__n_components=391, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'RAW') 
[CV]  reduce_dim__n_components=231, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), score=0.6363636363636364, total=   2.3s
[CV]  reduce_dim__n_components=231, filter__columns=None, combining__columns=('DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', 'LS'), score=0.6666666666666666, total=   2.3s
[CV]  reduce_dim__n_components=201, filter__c

[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  4.3min finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
          error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('combining', CombineMatrixTransformer(column_name='combined', columns=None,
             op=<function mean at 0x7fc76825a840>)), ('spread_out', SpreadOutMatrixTransformer(columns=['combined'], symmetric=True)), ('filter', FeatureMatrixTransformer(columns=...om_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classify', GaussianNB(priors=None))]),
          fit_params=None, iid=True, n_iter=150, n_jobs=-1,
          param_distributions={'combining__columns': [('DTI_FA',), ('DTI_L1',), ('DTI_MD',), ('DTI_RX',), ('RAW',), ('LS',), ('DTI_FA', 'DTI_L1'), ('DTI_FA', 'DTI_MD'), ('DTI_FA', 'DTI_RX'), ('DTI_FA', 'RAW'), ('DTI_FA', 'LS'), ('DTI_L1', 'DTI_MD'), ('DTI_L1', 'DTI_RX'), ('DTI_L1', 'RAW'), ('DTI_L1', 'LS'), (..., 'DTI_RX', 'RAW', 'LS')], 'filter__columns': [None], 'reduce_dim__n_components': range(1, 500, 1

In [11]:
%run ../grid_results.ipynb

reduce_dim__n_components = 1
filter__columns = None
combining__columns = ('DTI_FA', 'DTI_MD', 'DTI_RX', 'RAW')
Test
Accuracy in test: 60.61%
             precision    recall  f1-score   support

          0       0.68      0.71      0.70        21
          1       0.45      0.42      0.43        12

avg / total       0.60      0.61      0.60        33

Train
Accuracy in train: 70.77%
             precision    recall  f1-score   support

          0       0.73      0.88      0.80        42
          1       0.64      0.39      0.49        23

avg / total       0.70      0.71      0.69        65

